In [38]:
from google.cloud import bigquery as bq

In [14]:
!sudo chown -R jupyter:jupyter /home/jupyter/JSR_NYCTaxi

In [11]:
BUCKET = 'nyctaxi-284709'
PROJECT = 'nyctaxi-284709'
REGION = 'us'

In [12]:
import os

os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

# Import Data, Determine Feature Columns, Unique Values of Each Feature, Range of Values for Each Feature, Missing Values.

- Focus on yellow cab data and ignore green and for hire vehicles since yellow cabs are most prevalent kind of taxi and their data go back the furthest.
- Download 2019 data from original source since this year is not included in Big Query public datasets. Download earlier years from Google Big Query public datasets since these won't waste storage space in my Google Cloud bucket and can be accessed directly from this notebook via Google Cloud AI Platform. 


In [70]:
from google.cloud import storage
import urllib.request

project_id = 'nyctaxi-284709'
bucket_name = 'nyctaxi-284709'
storage_client = storage.Client()

# function to upload individual csv data files from source url to google cloud for an individual file. adapted from https://stackoverflow.com/questions/54235721/transfer-file-from-url-to-cloud-storage#:~:text=It%20is%20not%20possible%20to,memory%2C%20or%20in%20a%20file.
def upload_blob(bucket_name, source_file_name, destination_blob_name):  
    file = urllib.request.urlopen(source_file_name)

    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_string(file.read(), content_type='csv')

# function to upload data files from source url to google cloud for a specified range of months and years 
def download_data(year_min, year_max): # not inclusive of year_max
    print('Transferring raw data from https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page to gs://nyctaxi-284709/')
    for year in range(year_min, year_max, 1):
        for month in range(1,13):
            print(f'Transferring Taxi Data for Year: {year}, Month: {month:02}')
            source_file_name = f'https://s3.amazonaws.com/nyc-tlc/trip+data/yellow_tripdata_{year}-{month:02}.csv'
            destination_blob_name = os.path.basename(source_file_name)
            upload_blob(bucket_name, source_file_name, destination_blob_name)

In [71]:
download_data(2019, 2020)

Transferring raw data from https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page to gs://nyctaxi-284709/
Transferring Taxi Data for Year: 2019, Month: 01
Transferring Taxi Data for Year: 2019, Month: 02
Transferring Taxi Data for Year: 2019, Month: 03
Transferring Taxi Data for Year: 2019, Month: 04
Transferring Taxi Data for Year: 2019, Month: 05
Transferring Taxi Data for Year: 2019, Month: 06
Transferring Taxi Data for Year: 2019, Month: 07
Transferring Taxi Data for Year: 2019, Month: 08
Transferring Taxi Data for Year: 2019, Month: 09
Transferring Taxi Data for Year: 2019, Month: 10
Transferring Taxi Data for Year: 2019, Month: 11
Transferring Taxi Data for Year: 2019, Month: 12


Inspect 2019 yellow cab data to get a sense of what features are included in each year's dataset. 

In [78]:
column_name = 'test'
query = f'''
SELECT {column_name} 
FROM bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2018
LIMIT 100
'''

In [79]:
query

'\nSELECT test \nFROM bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2018\nLIMIT 100\n'

### Get Distinct Values for Columns

In [217]:
# Construct a BigQuery client object.
client = bigquery.Client()

def get_bq_tablename(year):
    if year >= 2009 and year <=2016:
        table_name = f'bigquery-public-data.new_york.tlc_yellow_trips_{year}'
    elif year == 2017 or year == 2018:
        table_name = f'bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_{year}'
    elif year == 2019:
        table_name = f'nyctaxi-284709.tlc_yellow_trips_2019.tlc_yellow_trips_2019'
    else:
        print('Input to get_bq_tablename() must be a year between 2009 and 2019 inclusive')
        return
    return table_name

def get_column_names(year): # column names should be the same for all years, but 
    if year < 2009 and year > 2019:  
        print('Input to get_column_names() must be a year between 2009 and 2019 inclusive')
        return
    table_name = get_bq_tablename(year)
    dataset_name = '.'.join(table_name.split('.')[:-1])
    table =  table_name.split('.')[-1]
    query = f'''
    SELECT column_name FROM {dataset_name}.INFORMATION_SCHEMA.COLUMNS
    WHERE table_name = '{table}'
    '''
    return bq.Client().query(query).result().to_dataframe()

def get_distinct_values(column_name, year):
    table_name = get_bq_tablename(year)
    query = f'''
    SELECT DISTINCT {column_name} 
    FROM {table_name}
    '''
    return bq.Client().query(query).result().to_dataframe()
    
    

In [219]:
get_distinct_values('vendor_id', 2009)

,vendor_id
0,DDS
1,CMT
2,VTS


In [210]:
for year in range(2009,2020):
    print(year)
    print(get_bq_tablename(year))

2009
bigquery-public-data.new_york.tlc_yellow_trips_2009
2010
bigquery-public-data.new_york.tlc_yellow_trips_2010
2011
bigquery-public-data.new_york.tlc_yellow_trips_2011
2012
bigquery-public-data.new_york.tlc_yellow_trips_2012
2013
bigquery-public-data.new_york.tlc_yellow_trips_2013
2014
bigquery-public-data.new_york.tlc_yellow_trips_2014
2015
bigquery-public-data.new_york.tlc_yellow_trips_2015
2016
bigquery-public-data.new_york.tlc_yellow_trips_2016
2017
bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2017
2018
bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2018
2019
nyctaxi-284709.tlc_yellow_trips_2019.tlc_yellow_trips_2019


In [216]:
get_column_names(2009)

dataset_name: bigquery-public-data.new_york
table: tlc_yellow_trips_2009


,column_name
0,vendor_id
1,pickup_datetime
2,dropoff_datetime
3,passenger_count
4,trip_distance
5,pickup_longitude
6,pickup_latitude
7,rate_code
8,store_and_fwd_flag
9,dropoff_longitude


In [188]:
query = f'''
    SELECT column_name FROM nyctaxi-284709.tlc_yellow_trips_2019.INFORMATION_SCHEMA.COLUMNS
    WHERE table_name = 'tlc_yellow_trips_2019'
    '''

bq.Client().query(query).result().to_dataframe()

,column_name
0,VendorID
1,tpep_pickup_datetime
2,tpep_dropoff_datetime
3,passenger_count
4,trip_distance
5,RatecodeID
6,store_and_fwd_flag
7,PULocationID
8,DOLocationID
9,payment_type
